In [1]:
import requests
import pandas as pd
import json
from requests.exceptions import HTTPError, JSONDecodeError

In [2]:
#ALPHA INTELLIGENCE
#covar_url=f'https://alphavantageapi.co/timeseries/running_analytics?SYMBOLS={AAPL},{IBM},{IBM},{IBM},{IBM}&RANGE=2month&INTERVAL=DAILY&OHLC=close&WINDOW_SIZE=20&CALCULATIONS=MEAN,STDDEV(annualized=True)&apikey=F5G3ECI5K968B229'

# companyoverview_url=f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey=F5G3ECI5K968B229'
#get marketcap dividend , 50daymoving average, 200day moving averafe

forex_url='https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&apikey=F5G3ECI5K968B229'

crypto_url='https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=BTC&to_currency=CNY&apikey=F5G3ECI5K968B229'
# can get realtime-data, plot prices on a chart and bubble points on markecap value

# F5G3ECI5K968B229 sjsu
# NQR2WJRB6F3LVPH2 madhuv
# OEW6DWI5S3P1B2N1 lm
#ECONOMIC INDICATORS
fed_interest_url='https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=monthly&apikey=OEW6DWI5S3P1B2N1'
#monthly data of interest rates to plot with non-farm payroll and unemployment rate

tyield_url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity=10year{3,5,10}&apikey=OEW6DWI5S3P1B2N1'

retail_sales_url = 'https://www.alphavantage.co/query?function=RETAIL_SALES&apikey=OEW6DWI5S3P1B2N1'

unempl_url = 'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=OEW6DWI5S3P1B2N1' # monthly data as %

nonfarm_payroll_url = 'https://www.alphavantage.co/query?function=NONFARM_PAYROLL&apikey=OEW6DWI5S3P1B2N1'#monthly total nonfarm

durable_url = 'https://www.alphavantage.co/query?function=DURABLES&apikey=OEW6DWI5S3P1B2N1'
cpi_url = 'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=OEW6DWI5S3P1B2N1'
crude_url = 'https://www.alphavantage.co/query?function=WTI&interval=monthly&apikey=OEW6DWI5S3P1B2N1'

In [4]:
def ticker_data(tick):
    tick_url=f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={tick}&apikey=OEW6DWI5S3P1B2N1'
    
    rc = requests.get(tick_url)
    tick_data = rc.json()
    
    df = pd.DataFrame([tick_data])

    # Convert columns to numeric if they contain numerical data
    numeric_columns = ['MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare',
                       'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM',
                       'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM',
                       'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE',
                       'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta',
                       '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding']

    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
    """# Convert columns to numeric if they contain numerical data
    numeric_columns = ['MarketCapitalization', '50DayMovingAverage','200DayMovingAverage','DividendPerShare', 'DividendYield']
    tick_df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    tick_df=pd.DataFrame(tick_data[['Symbol','MarketCapitalization','50DayMovingAverage','200DayMovingAverage','DividendPerShare','DividendYield']])"""
    return df

In [5]:
print(ticker_data('AAPL'))

  Symbol     AssetType       Name  \
0   AAPL  Common Stock  Apple Inc   

                                         Description     CIK Exchange  \
0  Apple Inc. is an American multinational techno...  320193   NASDAQ   

  Currency Country      Sector              Industry  ... EVToRevenue  \
0      USD     USA  TECHNOLOGY  ELECTRONIC COMPUTERS  ...        7.82   

  EVToEBITDA   Beta  52WeekHigh  52WeekLow  50DayMovingAverage  \
0      23.19  1.308       197.7     123.48              179.74   

   200DayMovingAverage  SharesOutstanding  DividendDate  ExDividendDate  
0               175.45        15552800000    2023-11-16      2023-11-10  

[1 rows x 46 columns]


In [14]:
def topgl():
    
    top_gl_url = 'https://www.alphavantage.co/query?function=TOP_GAINERS_LOSERS&apikey=OEW6DWI5S3P1B2N1'


    r = requests.get(top_gl_url)
    data = r.json()
    
    top_gainers_df = pd.DataFrame(data['top_gainers'])
    top_losers_df = pd.DataFrame(data['top_losers'])
    most_actively_traded_df = pd.DataFrame(data['most_actively_traded'])
    
    top_gainers_df['category'] = 'Top Gainers'
    top_losers_df['category'] = 'Top Losers'
    most_actively_traded_df['category'] = 'Most Actively Traded'

    # Concatenate the DataFrames vertically
    merged_df = pd.concat([top_gainers_df, top_losers_df, most_actively_traded_df], ignore_index=True)
    merged_df['change_percentage'] = merged_df['change_percentage'].replace('%', '', regex=True)

    # Convert columns to numeric if needed
    numeric_columns = ['price', 'change_amount', 'change_percentage', 'volume']
    merged_df[numeric_columns] = merged_df[numeric_columns].apply(pd.to_numeric, errors='coerce')
    
    return merged_df

In [4]:
def economic_indicators(unempl_url,nonfarm_payroll_url,fed_interest_url,durable_url,cpi_url,retail_sales_url,crude_url): 
    
    rc = requests.get(nonfarm_payroll_url)
    datafarm = rc.json()
    
    rc = requests.get(unempl_url)
    dataemp = rc.json()
    
    rc = requests.get(fed_interest_url)
    datafed = rc.json()
    
    rc = requests.get(durable_url)
    datadur = rc.json()
    
    rc = requests.get(cpi_url)
    datacpi = rc.json()
    
    rc = requests.get(retail_sales_url)
    dataretail = rc.json()
    
    rc = requests.get(crude_url)
    datawticrude = rc.json()
    

    print(datafarm)
    nonfarm = pd.DataFrame(datafarm['data'])
    unemploy = pd.DataFrame(dataemp['data'])
    fed_interest = pd.DataFrame(datafed['data'])
    durable = pd.DataFrame(datadur['data'])
    cpi = pd.DataFrame(datacpi['data'])
    retail_sales=pd.DataFrame(dataretail['data'])
    crude_wti=pd.DataFrame(datawticrude['data'])
    
    #treasury yield API data code
    treasury_df=[]
    typ=['2year', '5year', '10year']
    for p in typ:
        tyield_url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity={p}&apikey=OEW6DWI5S3P1B2N1'
        rc = requests.get(tyield_url)
        datatyld = rc.json()
        
        t_yield=pd.DataFrame(datatyld['data'])
        t_yield['category'] = f'Treasury {p} Yield'
        treasury_df.append(t_yield)
        
    t_yield_all = pd.concat(treasury_df)
    
    
    nonfarm['category'] = 'Non Farm Payrolls'
    unemploy['category'] = 'Unemployment Rate'
    fed_interest['category'] = 'FED Interest Rate'
    durable['category'] = 'Durable Goods Orders'
    cpi['category'] = 'Consumer Price Inflation'
    retail_sales['category'] = 'Retail Sales'
    crude_wti['category'] = 'Crude WTI'
    
    
    economic_df = pd.concat([nonfarm, unemploy, fed_interest,durable,cpi,retail_sales,crude_wti,t_yield_all])
    economic_df['date'] = pd.to_datetime(economic_df['date'])
    
    return pd.DataFrame(economic_df)
                        

In [5]:
moddf=(economic_indicators(unempl_url,nonfarm_payroll_url,fed_interest_url,durable_url,cpi_url,retail_sales_url,crude_url))

{'name': 'Total Nonfarm Payroll', 'interval': 'monthly', 'unit': 'thousands of persons', 'data': [{'date': '2023-10-01', 'value': '157984'}, {'date': '2023-09-01', 'value': '156918'}, {'date': '2023-08-01', 'value': '156392'}, {'date': '2023-07-01', 'value': '156022'}, {'date': '2023-06-01', 'value': '156905'}, {'date': '2023-05-01', 'value': '156279'}, {'date': '2023-04-01', 'value': '155369'}, {'date': '2023-03-01', 'value': '154440'}, {'date': '2023-02-01', 'value': '153983'}, {'date': '2023-01-01', 'value': '152839'}, {'date': '2022-12-01', 'value': '155344'}, {'date': '2022-11-01', 'value': '155642'}, {'date': '2022-10-01', 'value': '155041'}, {'date': '2022-09-01', 'value': '153809'}, {'date': '2022-08-01', 'value': '153285'}, {'date': '2022-07-01', 'value': '152875'}, {'date': '2022-06-01', 'value': '153217'}, {'date': '2022-05-01', 'value': '152291'}, {'date': '2022-04-01', 'value': '151449'}, {'date': '2022-03-01', 'value': '150411'}, {'date': '2022-02-01', 'value': '149606'},

In [6]:
moddftt = moddf.pivot(index='date', columns='category', values='value')


In [7]:
moddf

,date,value,category
0,2023-10-01,157984,Non Farm Payrolls
1,2023-09-01,156918,Non Farm Payrolls
2,2023-08-01,156392,Non Farm Payrolls
3,2023-07-01,156022,Non Farm Payrolls
4,2023-06-01,156905,Non Farm Payrolls
...,...,...,...
842,1953-08-01,2.95,Treasury 10year Yield
843,1953-07-01,2.93,Treasury 10year Yield
844,1953-06-01,3.11,Treasury 10year Yield
845,1953-05-01,3.05,Treasury 10year Yield


In [8]:
moddftt

category,Consumer Price Inflation,Crude WTI,Durable Goods Orders,FED Interest Rate,Non Farm Payrolls,Retail Sales,Treasury 10year Yield,Treasury 2year Yield,Treasury 5year Yield,Unemployment Rate
date,,,,,,,,,,
1913-01-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-03-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-04-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-05-01,9.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-06-01,305.109,70.25,324238.0,5.08,156905,612243.0,3.75,4.64,3.95,3.6
2023-07-01,305.691,76.07,261612.0,5.12,156022,605403.0,3.90,4.83,4.14,3.5
2023-08-01,307.026,81.39,290578.0,5.33,156392,628192.0,4.17,4.90,4.31,3.8


In [9]:
moddf.reset_index(inplace=True)

In [10]:
moddf['date'] = pd.to_datetime(moddf['date'])

In [111]:
moddf

category,date,Consumer Price Inflation,Durable Goods Orders,FED Interest Rate,Non Farm Payrolls,Unemployment Rate
0,1913-01-01,9.800,NaN,NaN,NaN,NaN
1,1913-02-01,9.800,NaN,NaN,NaN,NaN
2,1913-03-01,9.800,NaN,NaN,NaN,NaN
3,1913-04-01,9.800,NaN,NaN,NaN,NaN
4,1913-05-01,9.700,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1325,2023-06-01,305.109,324238.0,5.08,156905,3.6
1326,2023-07-01,305.691,261612.0,5.12,156022,3.5
1327,2023-08-01,307.026,290578.0,5.33,156392,3.8
1328,2023-09-01,307.789,308176.0,5.33,156918,3.8


In [11]:
moddf.to_csv('economic_indicators.csv', index=True)

In [12]:
moddftt.to_csv('economic_indicatorsvc.csv', index=True)

In [15]:
merged_df=topgl()

In [16]:
merged_df

,ticker,price,change_amount,change_percentage,volume,category
0,MINM,4.4800,3.5596,386.7449,74055310,Top Gainers
1,SRZNW,0.0437,0.0321,276.7241,31240,Top Gainers
2,AMPE,5.8500,3.8901,198.4846,19168071,Top Gainers
3,AUUDW,0.0340,0.0223,190.5983,362,Top Gainers
4,SNGX,0.9802,0.5648,135.9653,97456163,Top Gainers
5,RWODW,0.0400,0.0230,135.2941,1812,Top Gainers
6,AGBAW,0.0198,0.0108,120.0000,100,Top Gainers
7,BHIL+,0.0381,0.0201,111.6667,4157,Top Gainers
8,RFACR,0.1653,0.0843,104.0741,20,Top Gainers
9,GDNRW,0.0195,0.0094,93.0693,100,Top Gainers


In [17]:
merged_df.to_csv('day_topstocks.csv', index=True)

In [ ]:
companyoverview_url=f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey=F5G3ECI5K968B229'

In [19]:
moddftt

category,Consumer Price Inflation,Crude WTI,Durable Goods Orders,FED Interest Rate,Non Farm Payrolls,Retail Sales,Treasury 10year Yield,Treasury 2year Yield,Treasury 5year Yield,Unemployment Rate
date,,,,,,,,,,
1913-01-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-03-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-04-01,9.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-05-01,9.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-06-01,305.109,70.25,324238.0,5.08,156905,612243.0,3.75,4.64,3.95,3.6
2023-07-01,305.691,76.07,261612.0,5.12,156022,605403.0,3.90,4.83,4.14,3.5
2023-08-01,307.026,81.39,290578.0,5.33,156392,628192.0,4.17,4.90,4.31,3.8


In [20]:
moddftt.index

DatetimeIndex(['1913-01-01', '1913-02-01', '1913-03-01', '1913-04-01',
               '1913-05-01', '1913-06-01', '1913-07-01', '1913-08-01',
               '1913-09-01', '1913-10-01',
               ...
               '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01',
               '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01',
               '2023-09-01', '2023-10-01'],
              dtype='datetime64[ns]', name='date', length=1330, freq=None)